In [84]:
import pandas as pd
import math

Store the data in two lists that contains the different lines

In [85]:
with open('C:/Users/juan_/Desktop/programación/NLP_ML_DL_UdemyCourse/documents_for_course/Edgar_allan_poe.txt',encoding="utf-8", mode = "r") as doc:
    poe_dataset = doc.readlines()

In [86]:
with open('C:/Users/juan_/Desktop/programación/NLP_ML_DL_UdemyCourse/documents_for_course/Robert_frost.txt',encoding="utf-8", mode = "r") as doc:
    frost_dataset = doc.readlines()

Create a function that generates a dataframe for each author and split that dataframe into testing and training 

In [87]:
def divide_set(dataset):
    dataframe_set = pd.DataFrame({'Author': pd.Series(dtype='str'),'Poem': pd.Series(dtype='str')})
    for sentence in range(len(dataset)):
        list_row = ["Poe",dataset[sentence]]
        dataframe_set.loc[len(dataframe_set)] = list_row
        

    training_data = dataframe_set.sample(frac=0.8, random_state=25)
    testing_data = dataframe_set.drop(training_data.index)

    return training_data, testing_data


Generate with our function out training and testing datasets

In [88]:
poe_training_data, poe_testing_data = divide_set(poe_dataset)
frost_training_data, frost_testing_data = divide_set(frost_dataset)

Create a function that, given a specific dataframe, returns a word to index mapping for every word in the dataset

In [89]:
def get_vocabulary_size(dataset):
    number_of_states = {}
    counter = 0
    for index, row in dataset.iterrows():
        poem_line = row['Poem']
        for word in poem_line.split(" "):
            if word not in number_of_states:
                number_of_states[word] = counter
                counter +=1
    number_of_states["unknown_word"] = len(number_of_states) + 1
    return number_of_states

In [90]:
states_in_poe = get_vocabulary_size(poe_training_data)
print(len(states_in_poe))

1601


Now we have to turn every line of text in the training dataset into its respective series of integers from our mapping. 
Then we create a markov model for each of the classes

In [57]:
def create_skeleton_transitions_matrix(Training_dataset):
    number_of_states = get_vocabulary_size(Training_dataset)
    dictionary_dataframe = {}
    for word in number_of_states:
        dictionary_dataframe[word] = 1
    dataframe_estados = pd.DataFrame(dictionary_dataframe, index=number_of_states)

    return dataframe_estados

In [58]:
def create_transition_matrix(dataset):
    possible_states = get_vocabulary_size(dataset)
    dataframe_estados = create_skeleton_transitions_matrix(dataset)
    transitions_values = dataframe_estados.to_dict()
    number_of_transitions = 0
    number_of_appearances_in_dataset_of_state = 0
    
    for state in range(len(possible_states)):
        state_origin = possible_states[state]
        for state in range(len(possible_states)):
            state_transition = possible_states[state]
            for sentence in dataset:
                if state_origin in sentence:
                    number_of_appearances_in_dataset_of_state +=1
                split_sentence = sentence.split(" ")
                for word in range(len(split_sentence)-1):
                    if split_sentence[word] == state_origin and split_sentence[word+1] == state_transition:
                        number_of_transitions +=1
            numerator = number_of_transitions+1
            denominator = number_of_appearances_in_dataset_of_state+len(possible_states)
            transition_probability = numerator/denominator
            transitions_values[state_origin][state_transition] = transition_probability
            number_of_appearances_in_dataset_of_state = 0
            number_of_transitions = 0
    final_matrix = pd.DataFrame(transitions_values)

    return final_matrix

In [59]:
def initial_state_probability(sentence, training_dataset):
    number_of_initial_state_in_dataset = 0
    number_of_sentences_in_dataset = len(training_dataset)
    number_of_possible_states = len(get_vocabulary_size(training_dataset))
    splitted_sentence = sentence.split(" ")
    initial_word = splitted_sentence[0]
    for sentence in training_dataset:
        if sentence.startswith(initial_word):
            number_of_initial_state_in_dataset +=1
    numerator = number_of_initial_state_in_dataset+1
    denominator = number_of_sentences_in_dataset+number_of_possible_states
    probability_of_initial_state = numerator/denominator
        
    return probability_of_initial_state

In [60]:
def get_sentence_probability(dataset,sentence):
    possible_states = get_vocabulary_size(dataset)
    probability = initial_state_probability(sentence, dataset)
    transitions_matrix = create_transition_matrix(dataset)
    splitted_sentence = sentence.split(" ")
    for word in range(len(splitted_sentence)-1):
        if word != 0:
            if splitted_sentence[word] in possible_states and splitted_sentence[word-1] in possible_states:
                transition_probability = transitions_matrix[splitted_sentence[word]][splitted_sentence[word-1]]
            else:
                transition_probability = 1/len(possible_states)**2
            probability = probability * transition_probability
        else:
            pass
        
    return math.log(probability)

In [61]:
def classify_sentence(sentence):
    probability_poe = get_sentence_probability(poe_dataset,sentence)
    probability_frost = get_sentence_probability(frost_dataset,sentence)
    probability_poe = math.log(probability_poe)
    probability_frost = math.log(probability_frost)
    print(probability_poe)
    print(probability_frost)
    if probability_poe > probability_frost:
        print("El texto es de poe")
    else:
        print("El texto es de frost")

In [62]:
classify_sentence("""And travellers now within that valley,
Through the red-litten windows, see
Vast forms that move fantastically
To a discordant melody;
While, like a rapid ghastly river,
Through the pale door,
A hideous throng rush out forever,
And laugh --but smile no more.""")

KeyboardInterrupt: 